# Loading corpus và bộ xác suất đã tính ở file buidling model

In [1]:
import pickle

file1 = open('./trained_model/tags_of_tokens.txt', 'rb')
tags_of_tokens = pickle.load(file1)

file2 = open('./trained_model/bigram_tag_prob.txt', 'rb')
bigram_tag_prob = pickle.load(file2)

file3 = open('./trained_model/train_emission_prob.txt', 'rb')
train_emission_prob = pickle.load(file3)

In [2]:
tags_of_tokens

{'##': ['##'],
 'the': ['DT', 'JJ', 'IN', 'NNP'],
 'fulton': ['NNP'],
 'county': ['NNP', 'NN', 'NNPS'],
 'grand': ['NNP', 'JJ', 'FW'],
 'jury': ['NNP', 'NN'],
 'said': ['VBD', 'VB', 'VBN', 'JJ'],
 'friday': ['NNP'],
 'an': ['DT', 'NNP', 'CC'],
 'investigation': ['NN', 'NNP'],
 'of': ['IN', 'RP', 'NNP', 'RB'],
 'atlanta': ['NNP'],
 "'s": ['POS', 'VBZ', 'NNP', 'PRP', 'NNS', 'NNPS', 'IN', 'VBP'],
 'recent': ['JJ'],
 'primary': ['JJ', 'NN', 'NNP'],
 'election': ['NN', 'NNP'],
 'produced': ['VBD', 'VBN'],
 '``': ['``'],
 'no': ['DT', 'RB', 'UH', 'NNP', 'NN'],
 'evidence': ['NN', 'VBP'],
 "''": ["''", 'NN', 'SYM'],
 'that': ['IN', 'WP', 'DT', 'RP', 'WDT', 'RB', 'NNP', 'UH', 'PDT'],
 'any': ['DT', 'RB'],
 'irregularities': ['NNS'],
 'took': ['VBD'],
 'place': ['NN', 'VB', 'NNP', 'VBP', 'RB'],
 '.': ['.', 'RP'],
 '&&': ['&&'],
 'further': ['RB', 'JJ', 'JJR', 'VB', 'RBR', 'NNP'],
 'in': ['IN', 'RB', 'RP', 'RP|IN', 'NNP', 'NN', 'FW', 'VBD', 'IN|RP'],
 'term-end': ['JJ'],
 'presentments': ['NNS']

In [3]:
def Predict_tag(sent): 
    #Dùng để lưu giá trị tag tốt nhất ở mỗi từ
    store_tag = {}
    #ex: store_tag = {step_i.:{tag_1:[Tag của từ phía trước , giá trị của Tag]}}

    for i,word in enumerate(sent):

        #Đây là trường hợp dùng để gán xác suất thấp nhất cho các ký tự đặc biệt '##' đầu câu)
        if i == 1:
            store_tag[i] = {}

            tags = tags_of_tokens[word]
            for t in tags:
                try:
                    # Nếu từ trong tập test tồn tại trong train data
                    store_tag[i][t] = ['##',bigram_tag_prob['##'][t]*train_emission_prob[t][word]]
                except:
                    # '##' không nằm trong corpus của train data lúc này xác suất sẽ là 0.0001
                    store_tag[i][t] = ['##',0.0001]

        #Duyệt từ đầu tiên
        if i>1:
            store_tag[i] = {}

            #Load tag của word thứ i-1
            previous_tags = list(store_tag[i-1].keys())

            #Duyệt lần lượt các tag của từ thứ i
            current_tags  = tags_of_tokens[word]
            
            #tìm giá trị tốt nhất của tag i-1 so với các tag ở i và lưu vào store_tag
            for t in current_tags:                             
                temp = []
                for pt in previous_tags:                         
                    try:
                        temp.append(store_tag[i-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][word])
                    except:
                        temp.append(store_tag[i-1][pt][1]*0.0001)
                max_temp_index = temp.index(max(temp))

                #Postag tốt nhất tìm được
                best_pt = previous_tags[max_temp_index]
                store_tag[i][t]=[best_pt,max(temp)]

    #Quay lui để tìm đường có giá trị cao nhất hay các tag phù hợp cho các từ trong câu
    pred_tags = []
    total_steps_num = store_tag.keys()
    last_step_num = max(total_steps_num)
    
    for bs in range(len(total_steps_num)):
        step_num = last_step_num - bs
        
        if step_num == last_step_num:
            pred_tags.append('&&')
            print(store_tag[step_num]['&&'][0])
            pred_tags.append(store_tag[step_num]['&&'][0])
            
        if step_num<last_step_num and step_num>0:
            pred_tags.append(store_tag[step_num][pred_tags[len(pred_tags)-1]][0])

    return list(reversed(pred_tags))

#Để chạy một câu bắt buộc phải có ## và && để kết thúc

In [9]:
tokens='## He had his own face , dark and beautiful . &&'.lower().split()
Predict_tag(tokens)

.


['##', 'PRP', 'VBD', 'PRP$', 'JJ', 'NN', ',', 'JJ', 'CC', 'JJ', '.', '&&']

In [ ]:
tokens='## He had his own face , dark and beautiful . &&'.lower().split()
Predict_tag(tokens)